In [1]:
import sys

import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import importlib
from scipy.io import FortranFile

import time 
import cftime 

import RegridField as RgF

# import modules in other directories
sys.path.append('../Utils/')
import GridUtils as GrU
import MakePressures as MkP
import humiditycalcs as hum
import MyConstants as Con

sys.path.append('../Regridder/')
import Initialize as Prep
import ReadInSrc as RdSrc
import WriteDST as WrDst
import GenRegrid as GR
import latlon_w_scrip as LL
import var_A_x_B as AxB


importlib.reload( RgF )
importlib.reload( AxB )
importlib.reload( LL )
importlib.reload( GR )
importlib.reload( Prep )
importlib.reload( RdSrc )
importlib.reload( WrDst )


 a path added in VertRegridFlexLL /glade/work/juliob/PyRegridding/Regridder/../Utils 
 MyConstants in /glade/work/juliob/PyRegridding/Utils 
Using Flexible parallel/serial VertRegrid 
 a path added in VertRegridFlexLL /glade/work/juliob/PyRegridding/Regridder/../Utils 
 MyConstants in /glade/work/juliob/PyRegridding/Utils 
 MyConstants in /glade/work/juliob/PyRegridding/Utils 
 a path added in GenRegrid /glade/work/juliob/PyRegridding/Regridder/../Utils 
Using Flexible parallel/serial VertRegrid 
 a path added in VertRegridFlexLL /glade/work/juliob/PyRegridding/Regridder/../Utils 
 MyConstants in /glade/work/juliob/PyRegridding/Utils 
 a path added in var_A_x_B /glade/work/juliob/PyRegridding/Regridder/../Utils 
Using Flexible parallel/serial VertRegrid 
 a path added in VertRegridFlexLL /glade/work/juliob/PyRegridding/Regridder/../Utils 
 MyConstants in /glade/work/juliob/PyRegridding/Utils 
 a path added in var_A_x_B /glade/work/juliob/PyRegridding/Regridder/../Utils 
 a path added i

<module 'WriteDST' from '/glade/work/juliob/PyRegridding/Drivers/../Regridder/WriteDST.py'>

In [10]:
##########################

restF = '/glade/derecho/scratch/juliob/archive/c64_gwR2_ne30pg3_FMTHIST_topfix_oldCTL_qbotune_x03/rest/1997-01-01-00000/c64_gwR2_ne30pg3_FMTHIST_topfix_oldCTL_qbotune_x03.cam.i.1997-01-01-00000.nc'


#restF= '/glade/campaign/cgd/amp/aherring/restart/inic/ne120/ERA5_x_ne120np4_L58_rgC2_WO.aero.2000-01-01-00000.nc'

#restF ='/glade/derecho/scratch/juliob/archive/c64_gwR2_ne30pg3_FMTHIST_topfix_rdgres_x02/rest/1987-01-01-00000/c64_gwR2_ne30pg3_FMTHIST_topfix_rdgres_x02.cam.i.1987-01-01-00000.nc'
RestX = xr.open_dataset(restF) 

#eraF  ='/glade/campaign/cgd/amp/juliob/ERA5/ne120np4/L32/ERA5_x_ne120np4_L32_rgC2_WO.2000-01-01-00000.nc'

eraF  ='/glade/campaign/cgd/amp/juliob/ERA5/ne30np4/L93/ERA5_x_ne30np4_L93_rgC2_WO.2000-01-01-00000.nc'
eraX  = xr.open_dataset(eraF) 


RestXvars = list(RestX.variables) 
eraXvars  = list(eraX.variables) 

so4_a1 = RestX.so4_a1.values
so2    = RestX.SO2.values
ps     = RestX.PS.values


print( np.shape(so4_a1) )

(1, 93, 48602)


In [3]:
print('lev' in RestX.SO2.dims)

True


In [11]:
import RegridField as RgF
import copy
importlib.reload( RgF )

eraX_xtnd = copy.deepcopy( eraX )
ps     = eraX.PS.values

for fld in RestXvars:
    Q1 = (('lev' in RestX[fld].dims) or ('ilev' in RestX[fld].dims) )
    Q2 = (fld not in eraXvars )
    #print( fld, Q1, Q2 )
    if ( (Q1==True ) and ( Q2==True) ):
        #print( fld, RestX[fld].dims )
        aa = RestX[fld].values
        aa_z = aa #RgF.Vert(DstVgrid='L58', DstTZHkey='tzc', SrcVgrid='L93', xfld_Src=aa, ps_Src=ps )
        print( fld, RestX[fld].attrs , np.shape(aa_z) )

        
        Dar = xr.DataArray( data=aa_z, 
                            dims=('time','lev','ncol',),
                            attrs=RestX[fld].attrs ,)
        
        eraX_xtnd[fld] = Dar
                


Using Flexible parallel/serial VertRegrid 
 a path added in VertRegridFlexLL /glade/work/juliob/PyRegridding/Regridder/../Utils 
 MyConstants in /glade/work/juliob/PyRegridding/Utils 
CFC11 {'mdims': 1, 'units': 'kg/kg', 'mixing_ratio': 'dry', 'long_name': 'CFC11', 'cell_methods': 'time: point'} (1, 93, 48602)
CFC12 {'mdims': 1, 'units': 'kg/kg', 'mixing_ratio': 'dry', 'long_name': 'CFC12', 'cell_methods': 'time: point'} (1, 93, 48602)
CH4 {'mdims': 1, 'units': 'kg/kg', 'mixing_ratio': 'dry', 'long_name': 'CH4', 'cell_methods': 'time: point'} (1, 93, 48602)
CLDICE {'mdims': 1, 'units': 'kg/kg', 'mixing_ratio': 'wet', 'long_name': 'Grid box averaged cloud ice amount', 'cell_methods': 'time: point'} (1, 93, 48602)
CLDLIQ {'mdims': 1, 'units': 'kg/kg', 'mixing_ratio': 'wet', 'long_name': 'Grid box averaged cloud liquid amount', 'cell_methods': 'time: point'} (1, 93, 48602)
CO2 {'mdims': 1, 'units': 'kg/kg', 'mixing_ratio': 'dry', 'long_name': 'CO2', 'cell_methods': 'time: point'} (1, 93, 

In [12]:
# Make destination file name by replacing '_WO.'

#eraF  ='/glade/campaign/cgd/amp/juliob/ERA5/ne30np4/L32/ERA5_x_ne30np4_L32_rgC2_WO.2000-01-01-00000.nc'

eraF_xtnd = eraF.replace('_WO.', '_WO_aero.' )
print(eraF_xtnd)
eraX_xtnd.to_netcdf(  eraF_xtnd )

/glade/campaign/cgd/amp/juliob/ERA5/ne30np4/L93/ERA5_x_ne30np4_L93_rgC2_WO_aero.2000-01-01-00000.nc


In [ ]:
eraX_xtnd

In [ ]:
print(so2x.shape)
print(so2.shape)
print(pmid_Dst.shape)

In [ ]:
print( pmid_Dst[0,-5:,20000] )
print( pmid_Src[0,-5:,20000] )
print( ps[0,20000] )


In [ ]:
#plt.plot( pmid_Src[0,92,:] )
plt.plot( pmid_Dst[0,57,:] - pmid_Src[0,92,:] )


In [ ]:
plt.plot( so2x[0,:,20000],pmid_Dst[0,:,20000],'o' )
plt.plot( so2[0,:,20000],pmid_Src[0,:,20000],'x' )


In [ ]:
#Dst='ne240np4'
Dst='fv1x1'
Src='ne30np4'

DstInfo = GrU.gridInfo(Dst) #,Vgrid=DstVgrid)
dstHkey = DstInfo['Hkey']
dst_type =DstInfo['type']
dst_scrip =DstInfo['scrip']

SrcInfo = GrU.gridInfo(Src)
srcHkey = SrcInfo['Hkey']
src_type =SrcInfo['type']
src_scrip =SrcInfo['scrip']



In [ ]:
%%time
roog = HRF.regrid(Src=Src, Dst=Dst )

In [ ]:
%%time
soo=HRF.regrid(xfld_Src=so4_a1, Src=Src, Dst=Dst )

In [ ]:
%%time
soo2=HRF.regrid(xfld_Src=so4_a1, Src=Src, Dst=Dst, RegridObj_In=roog )

In [ ]:

lat_Dst,lon_Dst = GrU.latlon( scrip=dst_scrip, Hkey=dstHkey )
lat_Src,lon_Src = GrU.latlon( scrip=src_scrip, Hkey=srcHkey )


In [ ]:
cles = 1.e-9*np.linspace(0,10,21)
plt.tricontourf(lon_Src,lat_Src,so4_a1[0,80,:] ,levels=cles )
plt.colorbar()

In [ ]:
#plt.tricontourf(lon_Dst,lat_Dst,soo2[0,80,:] ,levels=cles )
plt.contourf(lon_Dst,lat_Dst,soo2[0,80,:] ,levels=cles )
plt.colorbar()

In [ ]:
shap='tc' #'tzyx'
print(shap[-2:])

In [ ]:
print( len(xoo) )

In [ ]:
yoo=None
print( xoo is None)